In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import re
import numpy as np
import pandas as pd
from scipy import stats

import gensim
import json


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import itertools

import keras

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Activation, GlobalAveragePooling1D, Flatten, Concatenate, Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.optimizers import SGD, RMSprop, Adagrad, Adam
from keras.preprocessing.text import one_hot, text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.vis_utils import plot_model

import fnmatch

import warnings

import string
from pathlib import Path
from random import shuffle
from ast import literal_eval
import statistics

warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [ ]:
!pip install nltk
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
wordnet_lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:

WINDOWS_SIZE = 10
labels=['none','mild','moderate','moderately severe', 'severe']
num_classes = len(labels)


In [ ]:
def text_to_wordlist(text, remove_stopwords=True, stem_words=False):    
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [wordnet_lemmatizer.lemmatize(w) for w in text if not w in stops ]
        text = [w for w in text if w != "nan" ]
    else:
        text = [wordnet_lemmatizer.lemmatize(w) for w in text]
        text = [w for w in text if w != "nan" ]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    
    text = re.sub(r"\<", " ", text)
    text = re.sub(r"\>", " ", text)
    
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [ ]:
def transcripts_to_dataframe(directory):
    rows_list = []
        
    filenames = os.listdir(directory)
    
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
        
    for filename in filenames:
        transcript_path = os.path.join(directory, filename)
        transcript = pd.read_csv(transcript_path, sep='\t')
        m = re.search("(\d{3})_TRANSCRIPT.csv", filename)
        if m:
            person_id = m.group(1)
            p = {}
            question = ""
            answer = ""
            lines = len(transcript)
            for i in range(0, lines):
                row = transcript.iloc[i]
                if (row["speaker"] == "Ellie") or (i == lines - 1):
                    p["personId"] = person_id
                    if "(" in str(question):
                        question = question[question.index("(") + 1:question.index(")")]
                    p["question"] = question
                    p["answer"] = answer
                    if question != "":
                        rows_list.append(p)
                    p = {}
                    answer = ""
                    question = row["value"]
                else:
                    answer = str(answer) + " " + str(row["value"])

    all_participants = pd.DataFrame(rows_list, columns=['personId', 'question', 'answer'])
    all_participants.to_csv(directory + 'all.csv', sep=',')
    print("File was created")
    return all_participants

In [ ]:
#loading the data
data_path = "/content/drive/My Drive/transcripts/"
all_participants = transcripts_to_dataframe(data_path)

File was created


In [ ]:
all_participants_mix = all_participants.copy()
all_participants_mix['answer'] = all_participants_mix.apply(lambda row: text_to_wordlist(row.answer).split(), axis=1)

In [ ]:

words = [w for w in all_participants_mix['answer'].tolist()]
words = set(itertools.chain(*words))
vocab_size = len(words)
print(vocab_size)

7373


In [ ]:
windows_size = WINDOWS_SIZE
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(all_participants_mix['answer'])
tokenizer.fit_on_sequences(all_participants_mix['answer'])

all_participants_mix['t_answer'] = tokenizer.texts_to_sequences(all_participants_mix['answer'])

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/Depression_detect/model_glove_lstm_b.h5')

In [ ]:
def test_model(text, model):
    print(text)
    word_list = text_to_wordlist(text)
    list_of_words = word_list.split(" ")
    sequences = tokenizer.texts_to_sequences([word_list])
    word_tokens = sequences[0]
    size = len(word_tokens)
    test_phrases = []
    for i in range(size):
        tokens = word_tokens[i:min(i+windows_size,size)]  
        test_phrases.append(tokens)
    sequences_input = test_phrases
    sequences_input =  pad_sequences(sequences_input, value=0, padding="post", maxlen=windows_size)
    #print(sequences_input)
    
    predicted_classes = []
    for sequence in sequences_input:
        input_a = np.asarray([sequence])
        pred = model.predict(input_a, batch_size=64, verbose=0, steps=None)
        #print(pred)
        predicted_classes.append(np.argmax(pred))   
        
    freq = {} 
    for class_ in predicted_classes: 
        if (class_ in freq): 
            freq[class_] += 1
        else: 
            freq[class_] = 1
            
    #print(predicted_classes)
    
    for key, value in freq.items():
        #print(key)
        label = labels[key]
        percentage = value/len(predicted_classes) * 100
        
        print(label + ": " + str(percentage))

In [ ]:
sen = "All is going right with the party, I'm happy to know new people"
test_model(sen, model)
sen = "I want an ice cream and have some fries for lunch"
test_model(sen, model)
sen = "I'm afraid of losing my work, I don't have any money"
test_model(sen, model)
sen = "I'm worried about my future, I'm afraid of it"
test_model(sen, model)
sen = "I am a graduate student"
test_model(sen, model)
sen = "I am getting married"
test_model(sen, model)
sen = "This party is great, I know lots of people"
test_model(sen, model)
sen = "I miss my parents, brothers and sisters"
test_model(sen, model)
sen = "I detest my horrible job"
test_model(sen, model)
sen = "suicide"
test_model(sen, model)

All is going right with the party, I'm happy to know new people
mild: 100.0
I want an ice cream and have some fries for lunch
moderately severe: 60.0
mild: 40.0
I'm afraid of losing my work, I don't have any money
moderately severe: 33.33333333333333
mild: 33.33333333333333
none: 33.33333333333333
I'm worried about my future, I'm afraid of it
moderately severe: 14.285714285714285
mild: 85.71428571428571
I am a graduate student
mild: 100.0
I am getting married
mild: 50.0
moderately severe: 50.0
This party is great, I know lots of people
mild: 100.0
I miss my parents, brothers and sisters
mild: 25.0
moderately severe: 75.0
I detest my horrible job
mild: 100.0
suicide
mild: 100.0


In [ ]:
all_participants = pd.read_csv(data_path + 'all.csv', sep=',')

In [ ]:
answers_308 = all_participants[all_participants.personId==308].answer.tolist()
answers_308

[nan,
 nan,
 nan,
 nan,
 ' los angeles california',
 ' yes',
 ' um the southern california lifestyle the beaches the um active lifestyle',
 nan,
 ' the traffic the pretentious people',
 nan,
 ' um my teachers um',
 " i don't know certain teachers um got uh spiritual teachers that i um find a lot of guidance from",
 ' <laughter>',
 " that's it mm i like to skateboard and",
 " and just stay active stay healthy um anything that's um outdoors and active and healthy hiking biking rollerblading skateboarding all the above",
 nan,
 ' mm no',
 ' mm not really in a position to travel financially',
 nan,
 " if i could i'm sure i would enjoy seeing all the different cultures and places and meeting all the different people and all the interesting structures and uh the food so",
 nan,
 ' i meditate and i skateboard well i can meditate on my board',
 " mm i meditate it helps me relax um sometimes if i don't really feel like skating hard i'll just go for a nice uh sunset skate and meditate a nice two

In [ ]:
import math
sen = ""
for answer in answers_308:
    if type(answer)==str or not math.isnan(answer):
        sen+=answer
print(sen)

 los angeles california yes um the southern california lifestyle the beaches the um active lifestyle the traffic the pretentious people um my teachers um i don't know certain teachers um got uh spiritual teachers that i um find a lot of guidance from <laughter> that's it mm i like to skateboard and and just stay active stay healthy um anything that's um outdoors and active and healthy hiking biking rollerblading skateboarding all the above mm no mm not really in a position to travel financially if i could i'm sure i would enjoy seeing all the different cultures and places and meeting all the different people and all the interesting structures and uh the food so i meditate and i skateboard well i can meditate on my board mm i meditate it helps me relax um sometimes if i don't really feel like skating hard i'll just go for a nice uh sunset skate and meditate a nice two hour skate just relax mm it's not that hard you just gotta get to the skate spots you know i usually like skating um the

In [ ]:
test_model(sen, model)

 los angeles california yes um the southern california lifestyle the beaches the um active lifestyle the traffic the pretentious people um my teachers um i don't know certain teachers um got uh spiritual teachers that i um find a lot of guidance from <laughter> that's it mm i like to skateboard and and just stay active stay healthy um anything that's um outdoors and active and healthy hiking biking rollerblading skateboarding all the above mm no mm not really in a position to travel financially if i could i'm sure i would enjoy seeing all the different cultures and places and meeting all the different people and all the interesting structures and uh the food so i meditate and i skateboard well i can meditate on my board mm i meditate it helps me relax um sometimes if i don't really feel like skating hard i'll just go for a nice uh sunset skate and meditate a nice two hour skate just relax mm it's not that hard you just gotta get to the skate spots you know i usually like skating um the